In [1]:
import datasets
ds = datasets.load_from_disk("./taco_cleaned")
ds

/Users/federico.cassano/.pyenv/versions/3.9.19/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question', 'solutions', 'starter_code', 'input_output', 'difficulty', 'raw_tags', 'name', 'source', 'tags', 'skill_types', 'url', 'Expected Auxiliary Space', 'time_limit', 'date', 'picture_num', 'memory_limit', 'Expected Time Complexity'],
    num_rows: 10039
})

In [2]:
# figure out format of tests
import json
import sys
sys.set_int_max_str_digits(0)
keys = set()
types = set()
has_fn_name = 0

for ex in ds:
    eio = ex["input_output"]
    eio = json.loads(eio)
    types.add(type(eio))
    ks = eio.keys()
    if "fn_name" in ks:
        has_fn_name += 1
    for k in ks:
        keys.add(k)

print(types)
print(keys)
print(has_fn_name)

{<class 'dict'>}
{'fn_name', 'inputs', 'outputs'}
544


In [2]:
9000 * 5

# MinHash + Locality Sentitive Hasking, Jaccard Similarity 0.5
# pick 5 dedup
# 1 with original prompt, 4 LLM to rephrase the prompt

45000

In [10]:
ds = ds.shuffle()

In [ ]:
import json
from tqdm import tqdm
from codeprm.execution import exec_io_test

subset = ds.select(range(10))
for i, ex in enumerate(subset):
    print(f"++ RUNNING {i}")
    for sol in tqdm(ex["solutions"][:5]):
        tests = json.loads(ex["input_output"])
        inputs = tests["inputs"]
        outputs = tests["outputs"]
        print(exec_io_test(sol, inputs, outputs))

In [3]:
import json
only_fn_name = ds.filter(lambda ex: "fn_name" in json.loads(ex["input_output"]).keys())

In [8]:
ex = ds[3]
tests = json.loads(ex["input_output"])
print(tests["outputs"])

['0\n', '0\n', '0\n', '1\n', '1\n', '0\n', '7\n', '0\n', '4\n', '4\n', '12\n', '0\n', '0\n', '17\n', '761\n', '0\n', '4\n', '2\n', '10\n', '0\n', '1\n', '0\n', '3\n', '758\n', '5\n', '9\n', '2\n', '772\n', '11\n', '1479\n', '1515\n', '1510\n', '4\n', '16\n', '795\n', '6\n', '774\n', '773\n', '1549\n', '1771\n', '15\n', '2658\n', '776\n', '768\n', '1533\n', '1543\n', '1783\n', '2662\n', '779\n', '1528\n', '1559\n', '1789\n', '2667\n', '1\n', '2\n', '1\n', '1\n', '1\n', '0\n', '2\n', '5\n', '1\n', '2\n', '1510\n', '0\n', '3\n', '6\n', '2\n', '1\n', '16\n', '5\n', '3\n', '2\n', '15\n', '6\n', '0\n', '1\n', '6\n']


In [9]:
ex = only_fn_name[3]
tests = json.loads(ex["input_output"])
print(tests["inputs"])

[[[1, 120], [2, 60], 4], [[1, 120], [2, 60], 8], [[1, 120], [4, 30], 8], [[1, 120], [3, 60], 9], [[1, 120], [3, 60], 27], [[1, 120], [9, 30], 27], [[1, 81], [2, 27], 4], [[1, 81], [2, 27], 8], [[1, 81], [4, 9], 8], [[1, 81], [5, 27], 25], [[1, 81], [5, 27], 125], [[1, 81], [25, 9], 125], [[4, 30], [2, 60], 1], [[5, 27], [1, 81], 1], [[4, 9], [8, 3], 1], [[1, 120], [1, 120], 1], [[4, 99], [4, 99], 4], [[9, 1], [9, 1], 9]]


In [ ]:
import json
from tqdm import tqdm
from codeprm.execution import exec_named_test

subset2 = only_fn_name.select(range(10))
for i, ex in enumerate(subset2):
    print(f"++ RUNNING {i}")
    for sol in tqdm(ex["solutions"][:5]):
        tests = json.loads(ex["input_output"])
        inputs = tests["inputs"]
        outputs = tests["outputs"]
        name = tests["fn_name"]
        print(exec_named_test(sol, inputs, outputs, name))

In [5]:
ds[10]["input_output"]

'{"fn_name": "make_acronym", "inputs": [["My aunt sally"], ["Please excuse my dear aunt Sally"], ["How much wood would a woodchuck chuck if a woodchuck could chuck wood"], ["Unique New York"], ["a42"], ["1111"], [64], [[]], [{}], [""]], "outputs": ["MAS", "PEMDAS", "HMWWAWCIAWCCW", "UNY", "Not letters", "Not letters", "Not a string", "Not a string", "Not a string", ""]}'

In [2]:
import datasets
import json
import os
ds = datasets.load_dataset(
    "BAAI/TACO", split="train+test", trust_remote_code=True)
no_sols = ds.filter(lambda ex: len(json.loads(ex["solutions"])) == 0, num_proc=os.cpu_count())
print(f"Number of no sols {len(no_sols)}")

Filter (num_proc=12): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26443/26443 [00:00<00:00, 37614.40 examples/s]


Number of no sols 5950


In [5]:
s = "ABCBC"
os = ds.filter(lambda ex: s in ex["question"])

In [7]:
print(os[1]["question"])

You are given a string s consisting of `A`, `B` and `C`.

Snuke wants to perform the following operation on s as many times as possible:

* Choose a contiguous substring of s that reads `ABC` and replace it with `BCA`.



Find the maximum possible number of operations.

Constraints

* 1 \leq |s| \leq 200000
* Each character of s is `A`, `B` and `C`.

Input

Input is given from Standard Input in the following format:


s


Output

Find the maximum possible number of operations.

Examples

Input

ABCABC


Output

3


Input

C


Output

0


Input

ABCACCBABCBCAABCB


Output

6
